In [1]:
from sentence_transformers import SentenceTransformer, util
import torch
import pandas as pd
import numpy as np
from concurrent.futures import ThreadPoolExecutor

In [2]:
# Define the model

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

In [3]:
def generate_embeddings(text):
    if not text:
        return []  # Return an empty list if input is empty or None
    embeddings = model.encode(text, convert_to_tensor=True)
    return embeddings

In [4]:
tv_data = pd.read_pickle('TV/tv_data_processed_part01.pkl')

In [5]:
tv_data = tv_data['Passages']

In [6]:
size = tv_data.shape[0]

In [ ]:
j = 6000
file_count = 2
for i in range(4000, size, 2000):
    print(i,j)
    if j >= size:
        current_data = tv_data[i:]
        embeddings_list = current_data.apply(generate_embeddings)
        
    else:
        current_data = tv_data[i:j]
        embeddings_list = current_data.apply(generate_embeddings)
        
    j +=2000
    if file_count < 10:
        file_name = 'TV/tv_embeddings_0' + str(file_count) + 'part.npz'
    else:
        file_name = 'TV/tv_embeddings_' + str(file_count) + 'part.npz'
    file_count += 1
        
    np.savez(file_name, embeddings_list.values, allow_pickle=True)

2000 4000
4000 6000


In [ ]:
import os

curr_dir = os.getcwd()
work_dir = curr_dir + '/TV/'

In [ ]:
npz_files = [file for file in os.listdir(work_dir) if file.endswith('.npz')]

all_data = []

for npz_file in sorted(npz_files):
    curr_data = np.load(os.path.join(work_dir, npz_file), allow_pickle=True)
    all_data.append(curr_data['arr_0'])
        
file_name = 'TV/tv_embeddings_part01.npz'
np.savez(file_name, all_data)



In [ ]:
npz_files = [file for file in os.listdir(work_dir) if file.endswith('part.npz')]
for npz_file in npz_files:
    os.remove(os.path.join(work_dir, npz_file))